In [2]:
import requests

url = "https://behnam-torob-ai-shopping-assistant.darkube.app/chat"
data = {
    "chat_id": "1",
    "messages": [{"type": "text", "content": "سلام، باحال ترین کالای ترب چیه؟"}]
}

# Disable system proxy
resp = requests.post(url, json=data, proxies={"http": None, "https": None})
print(resp.json())

{'message': 'باهال ترین کالای ترب در حال حاضر "سرخ کن فیلیپس مدل HD9270 ظرفیت 6.2 لیتر بدون روغن" است. این محصول از برند معتبر فیلیپس است و قیمت آن در فروشگاه\u200cهای مختلف بین 5,470,000 تا حدود 74,000,000 تومان متغیر است. تصویری از محصول نیز موجود است که می\u200cتوانید ببینید.', 'base_random_keys': ['jvnqdp'], 'member_random_keys': None}


In [5]:
dict(resp)

ValueError: dictionary update sequence element #0 has length 128; 2 is required